In [17]:
# !pip install qiskit_nature
# !pip install pyscf
# !pip install rdkit
#!pip install qiskit_aer

In [8]:
import pandas as pd
url = 'https://raw.githubusercontent.com/TastelessCandyzzz/Quantum_Chem_Hackathon/main/bace.csv'
d = pd.read_csv(url)
print(d.head())

                                                 mol     CID  Class  Model  \
0  O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...  BACE_1      1  Train   
1  Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...  BACE_2      1  Train   
2  S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...  BACE_3      1  Train   
3  S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...  BACE_4      1  Train   
4  S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...  BACE_5      1  Train   

      pIC50         MW   AlogP  HBA  HBD  RB  ...  PEOE6 (PEOE6)  \
0  9.154901  431.56979  4.4014    3    2   5  ...      53.205711   
1  8.853872  657.81073  2.6412    5    4  16  ...      73.817162   
2  8.698970  591.74091  2.5499    4    3  11  ...      70.365707   
3  8.698970  591.67828  3.1680    4    3  12  ...      56.657166   
4  8.698970  629.71283  3.5086    3    3  11  ...      78.945702   

   PEOE7 (PEOE7)  PEOE8 (PEOE8)  PEOE9 (PEOE9)  PEOE10 (PEOE10)  \
0      78.640335      226.85541      107.43491        3

In [9]:
df=d.loc[d["Model"]=='Test']
df

,mol,CID,Class,Model,pIC50,MW,AlogP,HBA,HBD,RB,...,PEOE6 (PEOE6),PEOE7 (PEOE7),PEOE8 (PEOE8),PEOE9 (PEOE9),PEOE10 (PEOE10),PEOE11 (PEOE11),PEOE12 (PEOE12),PEOE13 (PEOE13),PEOE14 (PEOE14),canvasUID
248,O=C(NCCC(C)(C)C)C(Cc1cc2cc(ccc2nc1N)-c1ccccc1C)C,BACE_276,1,Test,9.187087,403.55969,5.7644,2,2,7,...,84.122887,46.316166,247.789380,90.395477,37.133846,0.000000,7.980170,0.000000,0.000000,276
249,Fc1cc(cc(F)c1)CC(NC(=O)C(N1CCC(NC(=O)C)(C(CC)C...,BACE_277,1,Test,9.050610,615.73102,1.4277,5,5,13,...,67.960800,38.272877,304.052460,152.161880,34.923889,7.980170,32.336994,0.000000,24.663788,277
250,Fc1cc(ccc1C)CC(NC(=O)C)C(O)C[NH2+]C1CC2(Oc3ncc...,BACE_278,1,Test,9.004365,498.65250,3.3870,4,3,9,...,48.077168,49.532818,332.805330,84.453911,34.435734,15.387257,8.188327,0.000000,24.663788,278
251,Fc1cc(ccc1)CC(NC(=O)C)C(O)C[NH2+]C1CC2(Oc3ncc(...,BACE_279,1,Test,9.000000,484.62601,2.9008,4,3,9,...,48.077168,45.445873,299.932980,95.216072,34.435734,15.387257,8.188327,0.000000,24.663788,279
252,Fc1cc(cc(F)c1)CC(NC(=O)c1cc(cc(c1)/C(=N\OC)/C)...,BACE_280,1,Test,9.000000,639.75238,3.8163,6,3,17,...,37.771442,88.147522,261.311580,250.925540,35.014828,0.000000,23.571255,0.000000,24.663788,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,Clc1cc2nc(n(c2cc1)C(CC(=O)NCC1CCOCC1)CC)N,BACE_1543,0,Test,3.000000,364.86969,2.5942,3,2,6,...,37.212799,37.681076,180.226410,95.670128,30.107586,9.368159,7.980170,0.000000,0.000000,1543
1509,Clc1cc2nc(n(c2cc1)C(CC(=O)NCc1ncccc1)CC)N,BACE_1544,0,Test,3.000000,357.83731,2.8229,3,2,6,...,45.792797,47.349350,122.401500,99.877144,30.107586,9.368159,7.980170,0.000000,0.000000,1544
1510,Brc1cc(ccc1)C1CC1C=1N=C(N)N(C)C(=O)C=1,BACE_1545,0,Test,2.953115,320.18451,3.0895,2,1,2,...,47.790600,22.563574,96.290794,58.798935,20.071724,9.368159,0.000000,6.904104,0.000000,1545
1511,O=C1N(C)C(=NC(=C1)C1CC1c1cc(ccc1)-c1ccccc1)N,BACE_1546,0,Test,2.733298,317.38440,3.8595,2,1,3,...,77.219978,9.316234,95.907784,112.609720,20.071724,9.368159,0.000000,6.904104,0.000000,1546


In [10]:
df=pd.DataFrame(df,columns=["mol","Class"])
df

,mol,Class
248,O=C(NCCC(C)(C)C)C(Cc1cc2cc(ccc2nc1N)-c1ccccc1C)C,1
249,Fc1cc(cc(F)c1)CC(NC(=O)C(N1CCC(NC(=O)C)(C(CC)C...,1
250,Fc1cc(ccc1C)CC(NC(=O)C)C(O)C[NH2+]C1CC2(Oc3ncc...,1
251,Fc1cc(ccc1)CC(NC(=O)C)C(O)C[NH2+]C1CC2(Oc3ncc(...,1
252,Fc1cc(cc(F)c1)CC(NC(=O)c1cc(cc(c1)/C(=N\OC)/C)...,1
...,...,...
1508,Clc1cc2nc(n(c2cc1)C(CC(=O)NCC1CCOCC1)CC)N,0
1509,Clc1cc2nc(n(c2cc1)C(CC(=O)NCc1ncccc1)CC)N,0
1510,Brc1cc(ccc1)C1CC1C=1N=C(N)N(C)C(=O)C=1,0
1511,O=C1N(C)C(=NC(=C1)C1CC1c1cc(ccc1)-c1ccccc1)N,0


In [11]:
subset_df = df.sample(n=30, random_state=42).reset_index(drop=True)

In [12]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np

# Suppose you already have your subset_df
def smiles_to_fingerprint(smiles, radius=2, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)
    else:
        return np.zeros((n_bits,))

# Apply to all molecules
subset_df['features'] = subset_df['mol'].apply(smiles_to_fingerprint)

# Convert feature column into a NumPy matrix
X = np.stack(subset_df['features'])
Y = subset_df['Class'].values

[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerator
[06:15:10] DEPRECATION WARNING: please use MorganGenerat

In [13]:
from sklearn.decomposition import PCA

pca = PCA(n_components=4)  # 4 qubits → 4 input features
X_reduced = pca.fit_transform(X)

print("Reduced feature shape:", X_reduced.shape)

Reduced feature shape: (30, 4)


In [14]:
import os
import numpy as np

# RDKit for geometry generation from SMILES
from rdkit import Chem
from rdkit.Chem import AllChem

# --- Corrected Qiskit Nature Imports ---
# For specifying units (e.g., Angstrom)
from qiskit_nature.units import DistanceUnit
# The main driver to interface with chemistry software (PySCF)
from qiskit_nature.second_q.drivers import PySCFDriver
# Transformers to simplify the problem (e.g., freeze core electrons)
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
# The central problem class for electronic structure calculations
from qiskit_nature.second_q.problems import ElectronicStructureProblem
# Mappers to convert from fermionic operators to qubit operators
from qiskit_nature.second_q.mappers import JordanWignerMapper
# The standard class for representing Pauli operators
from qiskit.quantum_info import SparsePauliOp


# ---------- Helper: SMILES -> 3D XYZ (RDKit + MMFF) ----------
def smiles_to_xyz(smiles, random_seed=42):
    """
    Returns a multiline XYZ string (no header) with coordinates in Angstrom.
    Uses RDKit to embed the molecule in 3D and optimizes with the MMFF force field.
    """
    # Add explicit hydrogens
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    if mol is None:
        raise ValueError("Invalid SMILES string provided.")

    # Embed the molecule in 3D space
    AllChem.EmbedMolecule(mol, randomSeed=random_seed)
    # Optimize the geometry using MMFF
    AllChem.MMFFOptimizeMolecule(mol)

    conf = mol.GetConformer()
    lines = []
    for atom in mol.GetAtoms():
        p = conf.GetAtomPosition(atom.GetIdx())
        lines.append(f"{atom.GetSymbol()} {p.x:.8f} {p.y:.8f} {p.z:.8f}")
    return "\n".join(lines)


# ---------- Build PySCFDriver with a given basis and unit ----------
def build_pyscf_driver_from_xyz(xyz_str, basis='sto3g', charge=0, multiplicity=1):
    """
    Builds a Qiskit Nature PySCFDriver from a molecule's XYZ coordinates.
    """
    # The driver expects an 'atom' string in the format: 'H 0 0 0; H 0 0 0.74'
    atom_spec = "; ".join(xyz_str.strip().splitlines())

    driver = PySCFDriver(
        atom=atom_spec,
        unit=DistanceUnit.ANGSTROM,  # Correctly uses the imported UnitsType
        basis=basis,
        charge=charge,
        spin=(multiplicity - 1) # PySCF uses spin = 2S = (multiplicity - 1)
    )
    return driver


# ---------- Example: produce qubit Hamiltonian (Updated Workflow) ----------
def molecule_to_qubit_hamiltonian(smiles,
                                  basis='sto3g',
                                  active_electrons=None,
                                  active_orbitals=None,
                                  freeze_core=False):
    """
    A full pipeline from SMILES string to a qubit Hamiltonian (SparsePauliOp).

    Returns: A dictionary containing the results.
    """
    # 1) SMILES -> XYZ
    xyz = smiles_to_xyz(smiles)

    # 2) PySCFDriver -> Run calculation
    driver = build_pyscf_driver_from_xyz(xyz, basis=basis)
    es_problem = driver.run()

    # 3) Optionally apply transformers to reduce the problem size
    if freeze_core:
        transformer = FreezeCoreTransformer()
        es_problem = transformer.transform(es_problem)

    # if active_electrons is not None and active_orbitals is not None:
    #     # Use `num_electrons` and `num_molecular_orbitals` instead of the old keywords
    #     transformer = ActiveSpaceTransformer(
    #         num_electrons=active_electrons,
    #         num_molecular_orbitals=active_orbitals
    #     )
    #     es_problem = transformer.transform(es_problem)

    # 4) Get the second-quantized electronic Hamiltonian
    second_q_ops = es_problem.second_q_ops()
    second_q_hamiltonian = second_q_ops[0]

    # 5) Map the fermionic Hamiltonian to a qubit Hamiltonian
    #    The QubitConverter is no longer needed. The mapper does it all.
    mapper = JordanWignerMapper()
    qubit_op = mapper.map(second_q_hamiltonian)

    return {
        'qubit_operator': qubit_op,
        'num_spin_orbitals': es_problem.num_spin_orbitals,
        'num_particles': es_problem.num_particles,
        'n_qubits': qubit_op.num_qubits,
        'es_problem': es_problem,
        'xyz': xyz
    }

# ----------------- Simple run example -----------------
if __name__ == "__main__":
    # Using Aspirin as the test molecule
    test_smiles = "CCO"
    print(f"Building qubit Hamiltonian ({test_smiles})")

    # Define an active space to make the problem smaller and tractable
    # For a real calculation, this choice requires chemical insight.
    output = molecule_to_qubit_hamiltonian(
        test_smiles,
        basis='sto3g',
        active_electrons=2,  # e.g., the two electrons in the HOMO-LUMO gap
        active_orbitals=2,   # e.g., the HOMO and LUMO orbitals
        freeze_core=True     # Freeze the core electrons
    )

    print("-" * 50)
    print(f"Number of spin orbitals: {output['num_spin_orbitals']}")
    print(f"Number of particles (electrons): {output['num_particles']}")
    print(f"Number of qubits in mapped operator: {output['n_qubits']}")
    print("-" * 50)

    # Print a short excerpt of the resulting qubit operator
    print("Qubit operator (SparsePauliOp, trimmed):")
    # Convert to a list of strings and print the first 10 terms
    op_lines = str(output['qubit_operator']).splitlines()
    print("\n".join(op_lines[:10]))
    if len(op_lines) > 10:
        print("...")

Building qubit Hamiltonian (CCO)
--------------------------------------------------
Number of spin orbitals: 36
Number of particles (electrons): (10, 10)
Number of qubits in mapped operator: 36
--------------------------------------------------
Qubit operator (SparsePauliOp, trimmed):


In [ ]:
# Apply the function to each SMILES string in the subset_df
subset_df['qubit_hamiltonian'] = subset_df['mol'].apply(
    lambda smiles: molecule_to_qubit_hamiltonian(smiles, basis='sto3g', freeze_core=True)['qubit_operator']
)

# Display the updated DataFrame with the new 'qubit_hamiltonian' column
display(subset_df[['mol', 'Class', 'qubit_hamiltonian']].head())

In [42]:
# ============================================================
# quantum_descriptors.py
# Compute quantum descriptors (HF energy, VQE energy, etc.)
# directly from SMILES using PySCF + Qiskit Nature + Aer
# ============================================================

from rdkit import Chem
from rdkit.Chem import AllChem

# --- Qiskit Core Imports ---
from qiskit_aer import Aer
from qiskit import transpile
from qiskit_aer.primitives import EstimatorV2 as AerEstimator
from qiskit.circuit.library import EfficientSU2
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import AerSimulator
from qiskit.primitives import BackendEstimatorV2

# --- Qiskit Nature Imports ---
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer

# ============================================================
# Helper: Convert SMILES → PySCF geometry string
# ============================================================
def smiles_to_geometry(smiles: str):
    """Convert SMILES string to 3D geometry (Ångstrom)."""
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol)

    atoms = mol.GetAtoms()
    conf = mol.GetConformer()
    geom_str = "; ".join(
        f"{atoms[i].GetSymbol()} "
        f"{conf.GetAtomPosition(i).x:.4f} "
        f"{conf.GetAtomPosition(i).y:.4f} "
        f"{conf.GetAtomPosition(i).z:.4f}"
        for i in range(mol.GetNumAtoms())
    )
    return geom_str


# ============================================================
# Step 1: Convert geometry → qubit Hamiltonian
# ============================================================
def molecule_to_qubit_hamiltonian(geometry, basis="sto3g", active_electrons=2, active_orbitals=2):
    """Return qubit operator + electronic structure problem."""
    driver = PySCFDriver(atom=geometry, basis=basis, unit=DistanceUnit.ANGSTROM)
    es_problem = driver.run()

    # simplify with transformers
    freezer = FreezeCoreTransformer(freeze_core=True)
    es_problem = freezer.transform(es_problem)
    transformer = ActiveSpaceTransformer(
        num_electrons=active_electrons, num_spatial_orbitals=active_orbitals
    )
    es_problem = transformer.transform(es_problem)

    mapper = JordanWignerMapper()
    second_q_hamiltonian = es_problem.second_q_ops()[0]
    qubit_operator = mapper.map(second_q_hamiltonian)

    return {"qubit_operator": qubit_operator, "es_problem": es_problem}


# ============================================================
# Step 2: Compute VQE energy + Quantum Descriptors
# ============================================================
def compute_quantum_descriptors(
    smiles: str,
    basis: str = "sto3g",
    active_electrons: int = 2,
    active_orbitals: int = 2,
):
    """Compute key quantum descriptors from SMILES using VQE."""
    print(f"\n[•] Processing molecule: {smiles}")

    # → Convert SMILES to 3D geometry
    geometry = smiles_to_geometry(smiles)
    print(f"[i] Geometry: {geometry}")

    # → Build qubit Hamiltonian
    out = molecule_to_qubit_hamiltonian(geometry, basis, active_electrons, active_orbitals)
    qubit_op = out["qubit_operator"]
    es_problem = out["es_problem"]

    # Hartree–Fock baseline
    hf_energy = es_problem.reference_energy
    print(f"[+] Hartree–Fock energy: {hf_energy:.6f} Hartree")

    # → Define backend estimator + ansatz
    backend_sim = AerSimulator()
    estimator = BackendEstimatorV2(backend=backend_sim)

    ansatz = EfficientSU2(qubit_op.num_qubits, reps=2, entanglement="linear")
    ansatz = ansatz.decompose()

    # → Run VQE
    optimizer = COBYLA(maxiter=200)
    vqe_solver = VQE(estimator, ansatz, optimizer)
    vqe_result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

    vqe_energy = vqe_result.eigenvalue.real
    correlation_energy = hf_energy - vqe_energy

    # Transpile circuit for backend info (optional)
    transpiled_ansatz = transpile(ansatz, backend_sim)

    print(f"[i] Qubits used: {qubit_op.num_qubits}")
    print(f"[+] VQE ground-state energy: {vqe_energy:.6f} Hartree")
    print(f"[+] Correlation energy (HF – VQE): {correlation_energy:.6f} Hartree")

    # ========== Placeholder descriptors ==========
    # For now, these are mock placeholders (you’ll later compute via PySCF)
    homo_energy = -0.25  # Hartree (placeholder)
    lumo_energy = -0.10  # Hartree (placeholder)
    homo_lumo_gap = lumo_energy - homo_energy
    dipole_moment = 1.2  # Debye (placeholder)
    polarizability = 3.5  # Å³ (placeholder)

    # Collect descriptors
    descriptors = {
        "smiles": smiles,
        "hf_energy": hf_energy,
        "vqe_energy": vqe_energy,
        "correlation_energy": correlation_energy,
        "n_qubits": qubit_op.num_qubits,
        "homo_energy": homo_energy,
        "lumo_energy": lumo_energy,
        "homo_lumo_gap": homo_lumo_gap,
        "dipole_moment": dipole_moment,
        "polarizability": polarizability,
    }

    return descriptors


# ============================================================
# Example Run
# ============================================================
if __name__ == "__main__":
    test_smiles = "CCO"  # ethanol
    results = compute_quantum_descriptors(test_smiles, active_electrons=2, active_orbitals=2)
    print("\nFinal Quantum Descriptors:\n", results)



[•] Processing molecule: CCO
[i] Geometry: C 0.8782 -0.3015 -0.0263; C -0.3989 0.5283 -0.0741; O -1.5012 -0.2892 -0.3584; H 1.7476 0.3607 0.1704; H 1.0318 -0.8191 -0.9969; H 0.8120 -1.0576 0.7845; H -0.3053 1.2927 -0.8742; H -0.5444 1.0584 0.8944; H -1.7198 -0.7729 0.4807
[+] Hartree–Fock energy: -152.127989 Hartree


/tmp/ipython-input-3191825963.py:100: DeprecationWarning: The class ``qiskit.circuit.library.n_local.efficient_su2.EfficientSU2`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.efficient_su2 instead.
  ansatz = EfficientSU2(qubit_op.num_qubits, reps=2, entanglement="linear")


[i] Qubits used: 4
[+] VQE ground-state energy: -1.272919 Hartree
[+] Correlation energy (HF – VQE): -150.855070 Hartree

Final Quantum Descriptors:
 {'smiles': 'CCO', 'hf_energy': np.float64(-152.12798881397555), 'vqe_energy': np.float64(-1.2729185925894704), 'correlation_energy': np.float64(-150.8550702213861), 'n_qubits': 4, 'homo_energy': -0.25, 'lumo_energy': -0.1, 'homo_lumo_gap': 0.15, 'dipole_moment': 1.2, 'polarizability': 3.5}


In [ ]:
import pandas as pd
from tqdm import tqdm



# ---------- Step 2: Compute quantum features ----------
# Create empty lists to store descriptor results
hf_energies = []
vqe_energies = []
corr_energies = []
n_qubits_list = []
homo_list = []
lumo_list = []
gap_list = []
dipole_list = []
polarizability_list = []

for smiles in tqdm(subset_df["mol"], desc="Computing quantum descriptors"):
    try:
        descriptors = compute_quantum_descriptors(smiles)
        hf_energies.append(descriptors["hf_energy"])
        vqe_energies.append(descriptors["vqe_energy"])
        corr_energies.append(descriptors["correlation_energy"])
        n_qubits_list.append(descriptors["n_qubits"])
        homo_list.append(descriptors["homo_energy"])
        lumo_list.append(descriptors["lumo_energy"])
        gap_list.append(descriptors["homo_lumo_gap"])
        dipole_list.append(descriptors["dipole_moment"])
        polarizability_list.append(descriptors["polarizability"])
    except Exception as e:
        print(f"[!] Skipped molecule {smiles}: {e}")
        # Fill with NaN for skipped molecules
        hf_energies.append(None)
        vqe_energies.append(None)
        corr_energies.append(None)
        n_qubits_list.append(None)
        homo_list.append(None)
        lumo_list.append(None)
        gap_list.append(None)
        dipole_list.append(None)
        polarizability_list.append(None)

# ---------- Step 3: Add descriptors as new columns ----------
subset_df["hf_energy"] = hf_energies
subset_df["vqe_energy"] = vqe_energies
subset_df["correlation_energy"] = corr_energies
subset_df["n_qubits"] = n_qubits_list
subset_df["homo_energy"] = homo_list
subset_df["lumo_energy"] = lumo_list
subset_df["homo_lumo_gap"] = gap_list
subset_df["dipole_moment"] = dipole_list
subset_df["polarizability"] = polarizability_list

print("\n[✓] Quantum descriptors added successfully to subset_df!")
print(subset_df.head())

# ---------- Step 4: (Optional) Save updated dataset ----------
# subset_df.to_csv("bace_with_quantum_features.csv", index=False)
# print("[💾] Saved as bace_with_quantum_features.csv")


In [43]:
from tqdm import tqdm